# Import

In [52]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# Data

ref: [kaggle](https://www.kaggle.com/datasets/mazlumi/mbti-personality-type-twitter-dataset/code)

In [43]:
# df2 = pd.read_csv('./dataset/twitter_MBTI.csv').iloc[:,1:]

In [44]:
df2

,text,label
0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj
1,@Hispanthicckk Being you makes you look cute||...,intj
2,@Alshymi Les balles sont réelles et sont tirée...,intj
3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj
4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj
...,...,...
7806,"@sobsjjun God,,pls take care 😕|||@sobsjjun Hir...",intp
7807,@Ignis_02 wow last time i got intp https://t.c...,intp
7808,@akupilled A 100%|||@akupilled That SOMEONE wi...,entp
7809,If you’re #INTJ this one is for you | What is ...,infj


In [53]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


https://wikidocs.net/152922

https://dacon.io/codeshare/5619

In [73]:
inputs = tokenizer(df2.iloc[:,0],return_tensors="tf", padding=True, truncation=True)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).